In [ ]:
"""
    Fit a model cube to 3D data using the Python wrapper pyBBarolo.
    

    Example:
    
    path = '/Volumes/QbertPrimary/umdResearch/adapProposalNearby/pySpecKitCube/maybeRun3/3dBarolo_ngc2403/'
    fitsFile = path + 'examples/ngc2403.fits'
    hdu = fits.open(fitsFile)

    f3d = FitMod3D(fitsFile)
    f3d.init(radii=np.arange(15,1200,30),xpos=77,ypos=77,vsys=132.8,vrot=120,vdisp=8,vrad=0,z0=10,inc=60,phi=123.7)
    f3d.set_options(mask="SEARCH",free="VROT VDISP",wfunc=2,distance=3.2,outfolder=path+'test/')
    f3d.set_beam(bmaj=60,bmin=60,bpa=-80)
    %timeit bfrings, bestmod = f3d.compute(threads=4)
    # Make Plots
    f3d.plot_model()
"""

In [1]:
import os
import numpy as np
from astropy.io import fits
from astropy.wcs import WCS
from pyBBarolo import FitMod3D

from dictionaryThings import loadDict
from functionThings import waveToVel

In [2]:
# Size of the interpolated spaxels
arcsec = '1arc'
# Path to the highest level working directory.
topPath = '/Volumes/QbertPrimary/umdResearch/adapProposalNearby/'


# --------------------------- #
# Build the master dictionary #
# --------------------------- #
masterDict = dict()
dictList = ['objectInfoDict','emiLineDict','ratioDict']
for dictName in dictList:
    data = loadDict(dictFile= topPath + dictName + '.pkl')
    masterDict.setdefault(dictName,data)


# -------------------------------------------------- #
# Read in the parameter file containing line profile #
# velocity limits and continuum fitting information. #
# -------------------------------------------------- #
paramFile = np.genfromtxt(topPath + 'fittingParametersV4.txt',
                          dtype = None, autostrip = True, names = True, encoding=None)


In [3]:
masterDict['objectInfoDict']['m82']

{'1arcScaleBar': '175 pc',
 'centerCoo': (148.969687, 69.679383),
 'distance': 3.6,
 'inclination': 80,
 'majorPA': 65,
 'redshift': 0.000677}

In [4]:
for x in range(len(paramFile)):
    # Observation info.
    objectName =paramFile['objectName'][x]
    obsId = paramFile['obsId'][x]

    # Galaxy info.
    z = masterDict['objectInfoDict'][objectName]['redshift']
    raCenter = masterDict['objectInfoDict'][objectName]['centerCoo'][0]
    decCenter = masterDict['objectInfoDict'][objectName]['centerCoo'][1]
    inc = masterDict['objectInfoDict'][objectName]['inclination']
    majorPA = masterDict['objectInfoDict'][objectName]['majorPA']
    distance = masterDict['objectInfoDict'][objectName]['distance']

    # Line info.
    lineName = paramFile['lineNameShort'][x]
    restWave = masterDict['emiLineDict'][lineName]['restWave']
    # Line wavelength at the systemic velocity.
    sysWave = (1.+z)*restWave

    # Base for the object's file names.
    objectNameBase = str(obsId) + '_' + objectName + '_' + lineName
    
    # Directory to save 3dBarolo outputs.
    savePath = topPath + 'pySpecKitCube/maybeRun3/'+objectName+'/'+arcsec+'/3dbarolo/'+lineName+'/'
    if (not os.path.exists(savePath)):os.makedirs(savePath)

    break

In [5]:
    fitsFile = (topPath + 'pySpecKitCube/maybeRun3/'+objectName+'/'+arcsec+
               '/3dbarolo/fits/'+objectNameBase+'_hdrEditVel.fits')
    hdu = fits.open(fitsFile)
    hdr = hdu[0].header

    w = WCS(hdr).celestial
    xPos, yPos = w.wcs_world2pix(raCenter, decCenter, 1)
    vSys = int(waveToVel(sysWave, restWave))
    z0 = 8

In [6]:
    f3d = FitMod3D(fitsFile)

    f3d.init(radii = np.arange(2,20,2),
             xpos = int(xPos),
             ypos = int(yPos),
             vsys = vSys,
             vrot = 120,
             vdisp = 8,
             vrad = 0,
             z0 = 8,
             inc = inc,
             phi = majorPA)

    f3d.set_options(mask = 'SEARCH',
                    free = "VROT VDISP",
                    wfunc = 1,
                    distance = distance,
                    outfolder = savePath)

    f3d.set_beam(bmaj=10,bmin=10,bpa=110)

In [7]:
f3d.show_options()


Current options for 3DFIT task: -------------------------------------- 
 ltype        = 2          # Layer type along z  
 restwave     = -1.0       # Rest wavelenght of observed line  
 redshift     = -1.0       # Redshift of the galaxy  
 smooth       = True       # If false, disable smoothing  
 norm         = LOCAL      # Normalization type  
 cdens        = 10         # Surface density of clouds in a ring (1E20)  
 deltaphi     = 15.0       # Position angle variation (degrees)  
 twostage     = True       # Regularize and fit a second model  
 free         = VROT VDISP # Free parameters  
 outfolder    = /Volumes/QbertPrimary/umdResearch/adapProposalNearby/pySpecKitCube/maybeRun3/m82/1arc/3dbarolo/oiii88/ # Directory for outputs  
 distance     = 3.6        # Distance of the galaxy in Mpc  
 errors       = False      # Whether estimating errors  
 wfunc        = 1          # Weighting function for major axis  
 mask         = SEARCH     # Mask type  
 startrad     = 0          # 

In [8]:
    bfrings, bestmod = f3d.compute(threads=4)
    
    print objectName, lineName

m82 oiii88


In [9]:
    f3d.plot_model()

 Writing creative plots... Done.
